# Optimization with PyMoosh

Many physical and engineering problems require an optimization phase, in order to find an object or a structure that best fits a given constraint or objective.

To this end, the first step is to define the objective the structure must meet or approach. This can be as simple as  maximising or minimising a specific parameter (e.g. reflectivity or transmissivity at a given wavelength), or as complex as angularly separating polarizations or maximizing the amount of energy absorbed in a specific layer of the material.

In any case, this definition needs to be written as an **objective function**, that counts the proximity to the objective.

In the first example below, we simply want to design a dielectric mirror, so we want to maximise the reflectance on a given range of wavelengths.
For this, we will be using the *coefficient(...)* function that returns the reflectivity and transmitivity of a structure.

In [3]:
import PyMoosh as PM
import numpy as np

# The spectral range we are interested in
wav_beg = 500
wav_end = 800
nb_wav = 100
# The angle of incidence and polarization the structure is intended for
angle = 25 * np.pi / 180
polar = 0 # 0 for TE, 1 for TM


def objective_function(layers, wav_beg=wav_beg, wav_end=wav_end, nb_wav=nb_wav, angle=angle, polar=polar):
    """
    We want to maximise the reflectance of the structure over the whole spectrum
    """
    reflectivity = np.zeros(nb_wav)
    wav_list = np.linspace(wav_beg, wav_end, nb_wav)
    nb_lay = len(layers)//2
    mat = [1, 1.5, 2]
    stack = [0] + [1, 2] * nb_lay + [0]
    thickness = [0] + layers + [0]
    structure = PM.Structure(mat, stack, thickness, verbose=False)
    for i, wav in enumerate(wav_list):
        r, t, R, T = PM.coefficient(structure, wav, angle, polar)
        reflectivity[i] = R
    
    # Here we have computed all the reflectance spectrum
    # we want to maximise it, so we need to compute a value that
    # will be smaller when the reflectance is larger
    # There's 2 possible ways of doing this:
    # maximise the integral of the spectrum
    # -> cost = -np.sum(reflectivity)
    # maximise the smallest value of the spectrum
    # -> cost = 1 - np.min(reflectivity)
    
    return -np.sum(reflectivity)

## Optimization algorithms 

There are many different optimization algorithms out there. Luckily for you, we selected one that works particularly well for photonics: [Differential Evolution](https://en.wikipedia.org/wiki/Differential_evolution)

In its simplest form, it takes 4 arguments:
- The **objective function** you wrote
- The **budget**, or how many iterations it can make before stopping
- The **minimum value** of every parameter, defining the lower bound of parameter space
- The **maximum value** of every parameter, defining the upper bound of parameter space

It will output 2 values:
- The **best structure** in terms of objective function
- The **convergence** showing you how the cost of the best structures evolved throughout the algorithm

See how it goes:

In [5]:
budget = 1000 #

nb_layers = 10 # We want a 10-layer Dielectric mirror
min_lay = 10 # No layer thinner than 10 nm
max_lay = 800 # No layer thicker than 800 nm (the wavelength)

X_min = [min_lay] * nb_layers
X_max = [max_lay] * nb_layers

best, convergence = PM.Differential_Evolution(objective_function, budget, X_min, X_max)

AttributeError: module 'PyMoosh' has no attribute 'Differential_Evolution'